### Installing the Ultralytics YOLO package.

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.1/899.1 kB 16.0 MB/s eta 0:00:00


### Sets the current working directory to ConvertVideoToImage

In [2]:
import os

dir = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/'
os.chdir(dir)

### Read dates videos to convert from config file

In [3]:
import configparser

# Initialize the ConfigParser object
config = configparser.ConfigParser()
# Read the INI file
config.read('run_video_converter.ini')
# Dates to convert videos in
dates = config.get('General', 'dates').split(',')
# YOLO model path
yolo_path = config.get('General', 'yolo_path')

In [4]:
yolo_path

'..\\YoloDetector\\TrainedVersions\\Terns Detection 3.0.v3i.yolov8\\terns-detection\\weights\\best.pt'

### Get paths of videos to convert

In [5]:
import json

# Read the JSON configuration file
with open('tours_details.json', 'r', encoding='utf-8') as config_file:
    tour_configuration = json.load(config_file)
# Get videos dir path
videos_dir_path = tour_configuration["videos_dir"]

In [6]:
def isFileFromDates(path, dates):
    return any(date in path for date in dates)

def isVideoFile(path):
    video_extensions = ('.mp4', '.avi', '.mkv')
    return path.lower().endswith(video_extensions)

def get_dir_path_with_parent(directory_path):
  # Get the parent directory and the base name of the directory
  parent_dir, dir_name = os.path.split(os.path.normpath(directory_path))
  # Get the parent directory name and concatenate it with the directory name
  return os.path.join("/", os.path.basename(parent_dir), dir_name)[1:]

In [7]:
import glob

# Get all directories two levels under videos_dir
videos_names = glob.glob(os.path.join(videos_dir_path, '*', '*'))

# Filter out only videos from a specific date
videos_names = [get_dir_path_with_parent(path) for path in videos_names \
                  if isVideoFile(path) and isFileFromDates(path, dates)]

In [8]:
videos_names

['181/atlitcam181.stream_2024_06_11_10_00_01.mkv',
 '181/atlitcam181.stream_2024_06_11_15_00_02.mkv',
 '191/atlitcam191.stream_2024_06_11_15_00_02.mkv',
 '191/atlitcam191.stream_2024_06_11_10_00_04.mkv']

In [9]:
videos_names

['181/atlitcam181.stream_2024_06_11_10_00_01.mkv',
 '181/atlitcam181.stream_2024_06_11_15_00_02.mkv',
 '191/atlitcam191.stream_2024_06_11_15_00_02.mkv',
 '191/atlitcam191.stream_2024_06_11_10_00_04.mkv']

In [23]:
import subprocess

for video_name in videos_names:
    print(f'Converting video: {video_name}')
    images_relative_path_arg = f"-r {video_name}"  # Images dir to pass as argument
    command = f"python ./tours_extractor.py {images_relative_path_arg}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    # Check if the command ran successfully
    if result.returncode == 0:
        print(f'Output: {result.stdout}')
        print(f'Successfully video converted...')
    else:
        print(f'Error occurred while processing {video_name}')
        print(f'STDERR: {result.stderr}')

Converting video: 181/atlitcam181.stream_2024_06_11_10_00_01.mkv


KeyboardInterrupt: 

In [30]:
import os
import cv2
import json
import numpy as np

from PIL import Image


# Directory containing the final flag images and key areas coordinates JSON
FINAL_FLAG_IMAGES_DIR = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/FinalFlagSamples/'

# The TourExtractionValidator class validates the extraction process of camera tours,
# checking if the camera positions match the expected final flag positions.
class TourExtractionValidator:
    def __init__(self, flag_id,threshold=0.85):
        self._threshold = threshold

        # Load the key areas configuration from JSON
        with open(FINAL_FLAG_IMAGES_DIR + 'key_areas.json', 'r') as file:
            key_areas_data = json.load(file)

        self._key_areas = self._read_key_areas(key_areas_data, flag_id)


    # Reads and processes the key areas from the key_areas_data JSON
    def _read_key_areas(self, key_areas_data, flag_id):
        # Load grayscale final images for each camera
        final_images = {
            cam_id: cv2.imread(FINAL_FLAG_IMAGES_DIR + cam_id + '.jpg', cv2.IMREAD_GRAYSCALE)
            for cam_id in key_areas_data.keys()
        }
        # Map coordinates to cropped areas from final images
        return {
            cam_id: {
                # 'final_flag': flag_details['flag_id'],
                'final_flag': flag_id,
                'areas_coords': {
                    tuple(coordinates): self._cropImage(final_images[cam_id], *coordinates)
                    for coordinates in flag_details['coords']
                }
            }
            for cam_id, flag_details in key_areas_data.items()
        }


    # Crops a specific region from an image based on coordinates
    def _cropImage(self, img, x, y, w, h):
        return img[y:y + h, x:x + w]


    # Checks if the given image corresponds to the final flag for the specified camera
    def _is_final_flag(self, images_paths, cam_id):
        # Get key areas for the camera
        key_areas = self._key_areas[cam_id]['areas_coords']

        overall_diff = 0
        for image_path in images_paths:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                raise ValueError(f"Image not found: {image_path}")
            # Calculate the overall difference between key areas and the image
            for coords in key_areas:
                cropped1 = self._cropImage(image, *coords)
                cropped2 = key_areas[coords]
                # Calucalte distancce between cropped images
                diff_mat = np.maximum(cropped1, cropped2) - np.minimum(cropped1, cropped2)
                overall_diff += (np.mean(diff_mat) / 255) # 255 is the maximum value
                # print(overall_diff)

        similarity = (1 - (overall_diff / (len(key_areas) * len(images_paths))))

        print(f"Similarity: {similarity}")

        return similarity > self._threshold

    # Validates if the tour in the specified directory ends at the final flag
    def is_valid_tour(self, tour_dir):
        cam_id = None
        # Determine the camera ID from the directory name
        for flag in self._key_areas.keys():
            if flag in tour_dir:
                cam_id = flag

        if not cam_id:
            raise ValueError("tour_dir path is invalid.")

        if not os.path.exists(tour_dir):
            raise FileNotFoundError(f"Directory does not exist: {tour_dir}")

        # Get images related to the final flag from the tour directory
        last_flag_images = [tour_dir + image_name for image_name in os.listdir(tour_dir)  \
                 if str(self._key_areas[cam_id]['final_flag']) in image_name]

        # Return False if no images from the final flag are found
        if len(last_flag_images) == 0:
            return False

        # Validate the first and last images of the final flag
        images_edges = [last_flag_images[0], last_flag_images[-1]]

        return self._is_final_flag(images_edges, cam_id)

In [85]:
tour_dir = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/'

te_validator = TourExtractionValidator()

try:
    if not te_validator.is_valid_tour(tour_dir):
        print('deleting...', tour_dir)
        # utils_lib.delete_directory(tour_dir)
except Exception as e:
    print(f"Error: {e}")

Similarity: 0.6937768661271813
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_06_11_10_00_04/tour1/


In [31]:
tour_dir = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/'

try:
    # for flag_id in range(92, 139):
    for flag_id in range(24, 86):
        _te_validator = TourExtractionValidator(flag_id)
        print(flag_id)
        if not _te_validator.is_valid_tour(tour_dir):
            print('deleting...', tour_dir)
            # utils_lib.delete_directory(tour_dir)
except Exception as e:
    print(f"Error: {e}")

24
Similarity: 0.7997091891826661
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
25
Similarity: 0.7530432836499013
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
26
Similarity: 0.7198375899505101
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
27
Similarity: 0.7298238998396562
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
28
Similarity: 0.654770387613087
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
29
Similarity: 0.7269683592648404
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam191.stream_2024_05_30_10_00_00/tour1/
30
Similarity: 0.799709

In [22]:
tour_dir = '/content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/'

try:
    for flag_id in range(92, 139):
        _te_validator = TourExtractionValidator(flag_id)
        print(flag_id)
        if not _te_validator.is_valid_tour(tour_dir):
            print('deleting...', tour_dir)
            # utils_lib.delete_directory(tour_dir)
except Exception as e:
    print(f"Error: {e}")

92
Similarity: 0.7221413577919298
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
93
Similarity: 0.7717392848164907
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
94
Similarity: 0.8106014084408693
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
95
Similarity: 0.847600233227194
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
96
Similarity: 0.834385641165298
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
97
Similarity: 0.8344183611716292
deleting... /content/drive/MyDrive/tern_project/Eyal/ConvertVideoToImage/ImagesDir/atlitcam181.stream_2024_06_11_10_00_01/tour1/
98
Similarity: 0.8543124